In [ ]:
import pandas as pd
import os
import glob
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind

In [ ]:
df_client_demo = pd.read_csv("../data_files/clean/df_client_demo.csv")
df_experiment = pd.read_csv("../data_files/clean/df_experiment.csv")
df_web_data = pd.read_csv("../data_files/clean/df_web_data.csv")

In [ ]:
client_tenure_month_summary = df_client_demo['client_tenure_month'].describe()
client_tenure_month_summary

In [ ]:
client_age_summary = df_client_demo['client_age'].describe()
client_age_summary

In [ ]:
# Box Plot of Client Ages
plt.figure(figsize=(9, 6))
sns.boxplot(y=df_client_demo['client_age'], color='salmon')
plt.title('Box Plot of Client Ages')
plt.ylabel('Age')
plt.show()

In [ ]:
# Box Plot of Client Tenure Months
plt.figure(figsize=(9, 6))
sns.boxplot(y=df_client_demo['client_tenure_month'], color='cornflowerblue')
plt.title('Box Plot of Client Tenure Months')
plt.ylabel('Months')
plt.show()

In [ ]:
age_skewness_value = skew(df_client_demo['client_age'])
age_skewness_value
# This suggests that your distribution is nearly symmetrical, with no significant skew to either the left or right.
# The data is more or less evenly distributed around the mean, closer in shape to a normal distribution.

In [ ]:
age_kurtosis_value = kurtosis(df_client_demo['client_age'])
age_kurtosis_value
# This suggests that the distribution of your data has lighter tails and a flatter peak than a normal distribution.
# In practical terms, this means the data produces fewer extreme values or outliers compared to a normal distribution.

In [ ]:
tenure_month_skewness_value = skew(df_client_demo['client_tenure_month'])
tenure_month_skewness_value
# The distribution has a longer tail extending towards the higher values.
# This positive skewness suggests that there are outliers or a few high values that are pulling the mean to the right.

In [ ]:
tenure_month_kurtosis_value = kurtosis(df_client_demo['client_tenure_month'])
tenure_month_kurtosis_value
# The distribution has slightly heavier tails and a somewhat sharper peak than a normal distribution, indicating a moderate presence of outliers.
# The data is generally clustered around the mean more tightly than it would be in a normal distribution, with a few higher-than-usual deviations from the mean.

In [ ]:
# Calculate the Pearson correlation coefficient for age vs tenure months
correlation = df_client_demo['client_age'].corr(df_client_demo['client_tenure_month'])
print(f"Correlation between client_age and client_tenure_month: {correlation:.2f}")
# Although positive, a 0,31 coefficient reflects a no strong correlation

In [ ]:
# Create a scatter plot to visualize the relationship
plt.figure(figsize=(10, 6))
sns.scatterplot(x='client_age', y='client_tenure_month', data=df_client_demo, color='blue')
# Add a line for better visual of trend
sns.regplot(x='client_age', y='client_tenure_month', data=df_client_demo, scatter=False, color='red', ci=None)
# Add titles and labels
plt.title('Scatter Plot of Client Age vs. Client Tenure Month')
plt.xlabel('Client Age')
plt.ylabel('Client Tenure (Months)')
# Display the plot
plt.show()

In [ ]:
client_balance_summary = df_client_demo['balance'].describe()
client_balance_summary

We merge web data with final experimentt in order to have in the same df the client id linked to its variation (if it is control or test).

In [10]:
df_web_data_merged = pd.merge(df_web_data, df_experiment[['client_id', 'variation']], on='client_id', how='left')
df_web_data_merged.head()

,client_id,visitor_id,visit_id,process_step,date_time,variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test


In [ ]:
df_web_data_merged.info()

In [15]:
# After merging, we decided to drop all null values in the variation column since they are not part of the experiment.
df_web_data_merged_cleaned = df_web_data_merged.dropna(subset=['variation'])
df_web_data_merged_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 321309 entries, 0 to 637535
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     321309 non-null  int64 
 1   visitor_id    321309 non-null  object
 2   visit_id      321309 non-null  object
 3   process_step  321309 non-null  object
 4   date_time     321309 non-null  object
 5   variation     321309 non-null  object
dtypes: int64(1), object(5)
memory usage: 17.2+ MB


We split df into two df one for "Control" and other for "Test" users

In [17]:
web_data_merged_control = df_web_data_merged_cleaned[df_web_data_merged_cleaned['variation'] == 'Control']
web_data_merged_control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143462 entries, 13 to 637535
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     143462 non-null  int64 
 1   visitor_id    143462 non-null  object
 2   visit_id      143462 non-null  object
 3   process_step  143462 non-null  object
 4   date_time     143462 non-null  object
 5   variation     143462 non-null  object
dtypes: int64(1), object(5)
memory usage: 7.7+ MB


In [19]:
web_data_merged_test = df_web_data_merged_cleaned[df_web_data_merged_cleaned['variation'] == 'Test']
web_data_merged_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177847 entries, 0 to 637487
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     177847 non-null  int64 
 1   visitor_id    177847 non-null  object
 2   visit_id      177847 non-null  object
 3   process_step  177847 non-null  object
 4   date_time     177847 non-null  object
 5   variation     177847 non-null  object
dtypes: int64(1), object(5)
memory usage: 9.5+ MB


In [21]:
#check that we did not lose information in this transformation

total_cleaned = df_web_data_merged_cleaned.shape[0]
total_control = web_data_merged_control.shape[0]
total_test = web_data_merged_test.shape[0]
if total_cleaned == (total_control + total_test):
    print("Row counts match: ", total_cleaned)
else:
    print("Row counts do not match.")
    print(f"Total Cleaned: {total_cleaned}, Control + Test: {total_control + total_test}")

Row counts match:  321309


In [ ]:
web_data_merged_control.head(20)
web_data_merged_test.head(20)

In [23]:
# cast date_time as type = date
web_data_merged_control['date_time'] = pd.to_datetime(web_data_merged_control['date_time'])
web_data_merged_test['date_time'] = pd.to_datetime(web_data_merged_test['date_time'])

C:\Users\faval\AppData\Local\Temp\ipykernel_10748\447546305.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  web_data_merged_control['date_time'] = pd.to_datetime(web_data_merged_control['date_time'])
C:\Users\faval\AppData\Local\Temp\ipykernel_10748\447546305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  web_data_merged_test['date_time'] = pd.to_datetime(web_data_merged_test['date_time'])


In [25]:
# sort df by visit_id and date_time in order to check the duration
web_data_merged_control = web_data_merged_control.sort_values(by=["client_id", "visitor_id",'visit_id', "date_time"])
web_data_merged_test = web_data_merged_test.sort_values(by=["client_id", "visitor_id",'visit_id', "date_time"])

In [27]:
# We filter for visits with a visit_id that don't have a start step.
# These visits are considered to have started before the testing period,and thus, we exclude them from our analysis.
web_data_filtered_control = web_data_merged_control.groupby('visit_id').filter(lambda group: 'start' in group['process_step'].values)
web_data_filtered_test = web_data_merged_test.groupby('visit_id').filter(lambda group: 'start' in group['process_step'].values)

In [28]:
web_data_filtered_control = web_data_filtered_control.sort_values(by=["client_id", "visitor_id",'visit_id', "date_time"])
web_data_filtered_test = web_data_filtered_test.sort_values(by=["client_id", "visitor_id",'visit_id', "date_time"])

In [29]:
web_data_filtered_control.to_csv('web_data_filtered_control.csv', index=False)
web_data_filtered_test.to_csv('web_data_filtered_test.csv', index=False)